# Question 1

In [3]:
import numpy as np

In [4]:
#DONNÉES CONSOMMATION D'ÉNERGIE
SRAM_READ    = 7.95
SRAM_WRITE   = 5.45
BUFFER_READ  = 0.42
BUFFER_WRITE = 0.42
MAC          = 0.56

#DONNÉES HARDWARE
SRAM_LD      = 32768
SRAM_LW      = 8
BUFFER_LD    = 64
BUFFER_LW    = 8
MAX_LW       = 8

In [7]:
class energy_consumption:
    
    #Initialisation des données de la classe
    def __init__(self):
        self.total_sram_read    = 0
        self.total_sram_write   = 0
        self.total_buffer_read  = 0
        self.total_buffer_write = 0
        self.total_mac          = 0
   
    
    #Fonction pour sommer l'énergie utilisé par les opérations
    def counter(self, sram_read, sram_write, buffer_read, buffer_write, mac_operation):
        self.total_sram_read    += sram_read
        self.total_sram_write   += sram_write
        self.total_buffer_read  += buffer_read
        self.total_buffer_write += buffer_write
        self.total_mac          += mac_operation
    
    def disp_result(self):    
        energy_sram_read  = self.total_sram_read*SRAM_READ
        energy_sram_write = self.total_sram_write*SRAM_WRITE
        energy_buffer_read = self.total_buffer_read*BUFFER_READ
        energy_buffer_write = self.total_buffer_write*BUFFER_WRITE
        energy_mac = self.total_mac*MAC
        print("Nombre de lectures SRAM     :",self.total_sram_read)
        print("Nombre d'écriture SRAM      :",self.total_sram_write)
        print("Nombre de lectures buffer   :",self.total_buffer_read)
        print("Nombre d'écriture buffer    :",self.total_buffer_write)
        print("Nombre de d'opérations MAC  :",self.total_mac)
        print("Énergie de lecture SRAM     :",energy_sram_write,"pJ")
        print("Énergie de lecture SRAM     :",energy_sram_read,"pJ")
        print("Énergie d'écriture BUFFER   :",energy_buffer_read,"pJ")
        print("Énergie d'écriture BUFFER   :",energy_buffer_write,"pJ")
        print("Énergie d'opération MAC     :",energy_mac,"pJ")
        print("Consommation total          :",energy_sram_read+energy_sram_write+energy_buffer_read+energy_buffer_write+energy_mac,"pJ")
    
        
    def q1_ws(self):
        K1 = 16
        K0 = 2
        K = K1*K0
        P = 16
        R = 3
        for k1 in range(K1):   
            #Je peux loader tous les inputs en même temps car ils sont tous utilisé
            self.counter(P+R-1, 0, 0, P+R-1,0)
            for r in range(R):
                #Loading des poids: 2 à la fois en raison de la limite de K0
                self.counter(2, 0, 0, 2, 0)
                for k0 in range(K0):
                    
                    #Rien à faire, déja loadé
                    for p in range(P):
                        if r == 0:
                            self.counter(0, 0, 0,2,0) #Aller tout checher et écrire dans le buffer
                        else:
                            self.counter(0, 0, 1,1,0) #Juste besoin d'aller en chercher 1, car k et r ne sont pas update
                        self.counter(0, 0, 2,0,1) #Lecture ancien output, écriture output, mac
            self.counter(0,K0*P, 0,0,0) #Stocker les résultats
    def q1_os_u_1(self):
        K = 32
        P = 16
        R = 3
        for k in range(K):
            self.counter(P+R-1, 0, 0, P+R-1, 0) #Load les inputs
            self.counter(R, 0 , 0, R, 0) #Le nombre de poids est limité par R
            for p in range(P):
                for r in range(R):
                    if r == 0:
                        self.counter(0, 0, 0,2,0) #Aller tout checher et écrire dans le buffer
                    else:
                        self.counter(0, 0, 2,0,0) #Juste besoin d'aller en chercher 1, car k et r ne sont pas update
                    self.counter(0, 0, 1,1,1) #Lecture ancien output, écriture output, mac
            self.counter(0,16,0,0,0) #Stocker les résultats (grosseur du résultat du buffer est juste P)
            
    def q1_os_u_2(self):
        P = 16
        K = 32
        R = 3
        for p in range(P):
            self.counter(R,0,0,R,0) #Il ne peut y avoir que trois input à la fois dans le buffer
            for k in range(K):
                self.counter(R,0,0,R,0) #Le nombre de poids est limité par R
                for r in range(R):
                    if r == 0:
                        self.counter(0, 0, 0,2,0) #Aller tout checher et écrire dans le buffer
                    else:
                        self.counter(0, 0, 1,1,0) #Juste besoin d'aller en chercher 1, car k et r ne sont pas update
                    self.counter(0, 0, 2,0,1) #Lecture et mac
            self.counter(0,K,0,0,0) #Il n'y a qu'un p à la fois dans le buffer (lié au K)
                
    def q1_os_t_1(self):
        K1 = 4
        K0 = 8
        P  = 16
        R  = 3
        print(R*K0)
        for k1 in range(K1):
            self.counter(R+P-1, 0, 0, R+P-1, 0) #Tous les inputs peuvent être dans le buffer
            self.counter(R*K0, 0, 0, R*K0, 0) #Tous les poids dans le buffer
            for p in range(P):
                for k2 in range(K0):
                    for r in range(R):
                        if r == 0:
                            self.counter(0, 0, 0,2,0) #Aller tout checher et écrire dans le buffer
                        else:
                            self.counter(0, 0, 1,1,0) #Juste besoin d'aller en chercher 1, car k et r ne sont pas update
                        self.counter(0, 0, 2,0,1) #Lecture et mac
            self.counter(0,K0*P,0,0,0)
            
    def q1_os_t_2(self):
        P1 = 2
        P0 = 8
        R  = 3
        K  = 32
        for p1 in range(P1):
            self.counter(P0+R-1, 0,0, P0+R-1,0)
            for k in range(K):
                self.counter(R,0,0,R,0)
                for p0 in range(P0):
                    for r in range(R):
                        if r == 0:
                            self.counter(0, 0, 0,2,0) #Aller tout checher et écrire dans le buffer
                        else:
                            self.counter(0, 0, 1,1,0) #Juste besoin d'aller en chercher 1, car k et r ne sont pas update
                        self.counter(0, 0, 2,0,1) #Lecture et mac
            self.counter(0,P0*K,0,0,0) #Fait déborder le buffer? Peut-être mais là ça marche so j'y touche pu

In [6]:
weight_stationnary = energy_consumption()

weight_stationnary.q1_ws()
weight_stationnary.disp_result()

Nombre de lectures SRAM     : 384
Nombre d'écriture SRAM      : 512
Nombre de lectures buffer   : 4096
Nombre d'écriture buffer    : 2432
Nombre de d'opérations MAC  : 1536
Énergie de lecture SRAM     : 3052.8 pJ
Énergie d'écriture SRAM     : 2790.4 pJ
Énergie de lecture BUFFER   : 1720.32 pJ
Énergie d'écriture BUFFER   : 1021.4399999999999 pJ
Énergie d'opération MAC     : 860.1600000000001 pJ
Consommation total          : 9445.12 pJ


In [89]:
output_stationnary_1 = energy_consumption()
output_stationnary_1.q1_os_u_1()
output_stationnary_1.disp_result()

Nombre de lectures SRAM     : 672
Nombre d'écriture SRAM      : 512
Nombre de lectures buffer   : 3584
Nombre d'écriture buffer    : 3232
Nombre de d'opérations MAC  : 1536
Énergie de lecture SRAM     : 5342.400000000001 pJ
Énergie d'écriture SRAM     : 2790.4 pJ
Énergie de lecture BUFFER   : 1505.28 pJ
Énergie d'écriture BUFFER   : 1357.44 pJ
Énergie d'opération MAC     : 860.1600000000001 pJ
Consommation total          : 11855.680000000002 pJ


In [8]:
output_stationnary_2 = energy_consumption()
output_stationnary_2.q1_os_u_2()
output_stationnary_2.disp_result()

Nombre de lectures SRAM     : 1584
Nombre d'écriture SRAM      : 512
Nombre de lectures buffer   : 4096
Nombre d'écriture buffer    : 3632
Nombre de d'opérations MAC  : 1536
Énergie de lecture SRAM     : 2790.4 pJ
Énergie de lecture SRAM     : 12592.800000000001 pJ
Énergie d'écriture BUFFER   : 1720.32 pJ
Énergie d'écriture BUFFER   : 1525.44 pJ
Énergie d'opération MAC     : 860.1600000000001 pJ
Consommation total          : 19489.12 pJ


In [91]:
output_stationnary_t_1 = energy_consumption()
output_stationnary_t_1.q1_os_t_1()
output_stationnary_t_1.disp_result()

24
Nombre de lectures SRAM     : 168
Nombre d'écriture SRAM      : 512
Nombre de lectures buffer   : 4096
Nombre d'écriture buffer    : 2216
Nombre de d'opérations MAC  : 1536
Énergie de lecture SRAM     : 1335.6000000000001 pJ
Énergie d'écriture SRAM     : 2790.4 pJ
Énergie de lecture BUFFER   : 1720.32 pJ
Énergie d'écriture BUFFER   : 930.7199999999999 pJ
Énergie d'opération MAC     : 860.1600000000001 pJ
Consommation total          : 7637.2 pJ


In [92]:
output_stationnary_t_2 = energy_consumption()
output_stationnary_t_2.q1_os_t_2()
output_stationnary_t_2.disp_result()

Nombre de lectures SRAM     : 212
Nombre d'écriture SRAM      : 512
Nombre de lectures buffer   : 4096
Nombre d'écriture buffer    : 2260
Nombre de d'opérations MAC  : 1536
Énergie de lecture SRAM     : 1685.4 pJ
Énergie d'écriture SRAM     : 2790.4 pJ
Énergie de lecture BUFFER   : 1720.32 pJ
Énergie d'écriture BUFFER   : 949.1999999999999 pJ
Énergie d'opération MAC     : 860.1600000000001 pJ
Consommation total          : 8005.48 pJ


# Q2

In [ ]:
for k in range(K):
    
    for p in range(P):
        for r in range(R):